In [24]:
# imports
import numpy as np
from numpy import linalg as LA
from qiskit import QuantumCircuit
from qiskit.opflow.primitive_ops import PrimitiveOp
from qiskit.quantum_info import Operator
from qiskit.algorithms import NumPyEigensolver
from qiskit.utils import algorithm_globals
from qiskit.utils import QuantumInstance
from qiskit import Aer
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal

In [28]:
# the Hamiltonian

H1 = np.array([
    [0, 0, 0, 0], 
    [0, -1, 1, 0],
    [0, 1, -1, 0], 
    [0, 0, 0, 0]])

print("Hamiltonian")
print(H1)

H1_op = PrimitiveOp(H1)
print()
print(type(H1))
print()
print("is unitary:", Operator(H1).is_unitary())

Hamiltonian
[[ 0  0  0  0]
 [ 0 -1  1  0]
 [ 0  1 -1  0]
 [ 0  0  0  0]]

<class 'numpy.ndarray'>

is unitary: False


In [12]:
# classically compute min eigenvalue

H1_eigenvals = LA.eigvals(H1)
H1_min_eigenval = min(H1_eigenvals)

print("eigenvalues")
print(H1_eigenvals)
print()
print("min eigenvalue")
print(H1_min_eigenval)

eigenvalues
[ 0. -2.  0.  0.]

min eigenvalue
-2.0


In [13]:
# compute eigenvalues classically

npe = NumPyEigensolver()

H1_eigenvalues = npe.compute_eigenvalues(H1_op)

print(H1_eigenvalues)

{   'aux_operator_eigenvalues': None,
    'eigenstates': ListOp([VectorStateFn(Statevector([-2.77555756e-17-1.98408998e-17j,
              4.41702900e-01+5.52176193e-01j,
             -4.41702900e-01-5.52176193e-01j,
             -1.11022302e-16-2.77555756e-17j],
            dims=(2, 2)), coeff=1.0, is_measurement=False)], coeff=1.0, abelian=False),
    'eigenvalues': array([-2.])}


In [25]:
# compute min eigenvalue and eigenvector using qiskit's VQE 

seed = 50
algorithm_globals.random_seed = seed
qi = QuantumInstance(Aer.get_backend('statevector_simulator'), seed_transpiler=seed, seed_simulator=seed)

ansatz = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')

slsqp = SLSQP(maxiter=1000)

vqe = VQE(ansatz, optimizer=slsqp, quantum_instance=qi)

result = vqe.compute_minimum_eigenvalue(H1_op)

print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 68,
    'eigenstate': array([-1.62184660e-04+0.j, -7.07155717e-01+0.j,  7.07057805e-01+0.j,
        1.62211227e-04+0.j]),
    'eigenvalue': (-1.9999998851804786+0j),
    'optimal_parameters': {   ParameterVectorElement(θ[0]): 4.474004212975571,
                              ParameterVectorElement(θ[1]): 3.7552674802815225,
                              ParameterVectorElement(θ[2]): 0.4502124244654901,
                              ParameterVectorElement(θ[3]): 6.263984278482669,
                              ParameterVectorElement(θ[5]): 0.9943057528573237,
                              ParameterVectorElement(θ[4]): -2.5301216816670666,
                              ParameterVectorElement(θ[6]): -4.85880891119029,
                              ParameterVectorElement(θ[7]): 0.09117421264779375},
    'optimal_point': array([ 4.47400421,  3.75526748,  0.45021242,  6.26398428, -2.53012168,
        0.99430575, -4.85880891,  0.

In [14]:
# decompose the Hamiltonian into a sum of terms
# H1 = 0.5*(XX + YY + ZZ - II)

xx_qc = QuantumCircuit(2,2)
yy_qc = QuantumCircuit(2,2)
zz_qc = QuantumCircuit(2,2)
ii_qc = QuantumCircuit(2,2)

xx_qc.x(0)
xx_qc.x(1)

yy_qc.y(0)
yy_qc.y(1)

zz_qc.z(0)
zz_qc.z(1)

ii_qc.i(0)
ii_qc.i(1)


In [26]:
# ansatz using rx and ry gates 

def ansatz_rx_ry_prep(theta_x_0=np.pi/2, theta_y_0=np.pi/2, 
                      theta_x_1=np.pi/2, theta_y_1=np.pi/2 ):

    ansatz_rxry = QuantumCircuit(2)
    ansatz_rxry.rx(theta_x_0, 0)
    ansatz_rxry.ry(theta_y_0, 0)

    ansatz_rxry.rx(theta_x_1, 1)
    ansatz_rxry.ry(theta_y_1, 1)
    
    return ansatz_rxry    

In [27]:
# ansatz 

def ansatz_prep(theta):

    ansatz = QuantumCircuit(2)
    
    ansatz.h(0)
    ansatz.i(1)
    ansatz.cx(0,1)
    ansatz.rx(theta, 0)    
    ansatz.i(1)    
    
    return ansatz
    

In [ ]:
# compute energy for a given trial state

def compute_energy(ansatz):
    
    
    
    return energy

In [ ]:
# find minimum energy for a set of trial states

thetas = [0, np.pi/2, np.pi, (3/2)*np.pi, 2*np.pi]

for theta in thetas:
    


